# 👟 Behaviors

Behaviors extend the appearance and interactivity of the [2D](#2D-Graph) and
[3D](./3D.ipynb) graphs. These can affect [nodes](#Node-Behaviors),
[links](#Link-Behaviors) or certain aspects of the graph itself.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
from pathlib import Path

import ipyforcegraph.behaviors as B
import ipywidgets as W
import traitlets as T
from IPython.display import display

In [ ]:
with __import__("importnb").Notebook():
    import Shapes as S
    import Utils as U

## 2D Graph

> 💡 After rendering the cell below, select _Create New View For Output_ from the
> output's right-click menu to see more things added

In [ ]:
if __name__ == "__main__":
    fg, box = U.make_a_demo()
    box.description = "Behavior Demo"
    display(box)

> 💡 This demo will be incrementally updated: try _Create New View For Output_ from the
> context menu to watch.

## Data

A number of datasets are provided.

In [ ]:
def add_dataset(fg, box):
    from ipyforcegraph.sources.dataframe import DataFrameSource

    datasets = sorted(Path("datasets").glob("*.json"), key=lambda d: d.stat().st_size)
    picker = W.Select(
        options={d.name: d for d in datasets},
        multiple=False,
        rows=1,
        description="dataset",
    )

    def on_dataset_changed(change):
        old_source = fg.source
        dataset = json.loads(picker.value.read_text(encoding="utf-8"))
        fg.source = DataFrameSource(**dataset)
        old_source.close()

    picker.observe(on_dataset_changed)
    ui = U.make_a_collapsible_picker("graph: dataset", children={"": [picker]})
    box.graph_ui = {**box.graph_ui, "dataset": ui}

In [ ]:
if __name__ == "__main__":
    add_dataset(fg, box)

## Graph Features

In [ ]:
features = {
    "default_node_color": {"r": 31, "g": 120, "b": 179, "a": 1.0},
    "default_link_color": {"r": 66, "g": 66, "b": 66, "a": 0.5},
    "background_color": {"r": 0, "g": 0, "b": 0, "a": 0.0},
}


def add_graph_colors(fg, box):
    for feature, default_value in features.items():
        picker = U.make_an_rgba_picker(**default_value)
        ui = U.make_a_collapsible_picker(
            f"graph: {' '.join(feature.split('_'))}", children={"": [picker]}
        )
        T.dlink((picker, "color"), (fg, feature))
        box.graph_ui = {**box.graph_ui, feature: ui}

In [ ]:
if __name__ == "__main__":
    add_graph_colors(fg, box)

## Graph Behaviors

### `ImageCapture`

The current image can be streamed back to the kernel.

In [ ]:
def add_graph_image(fg, box):
    behavior = B.GraphImage()
    capturing = W.ToggleButton(description="capturing")
    frame_count = W.IntSlider()
    frames = W.HBox()
    ui = U.make_a_collapsible_picker(
        "graph: image",
        children={
            "capture": [
                W.VBox(
                    [
                        capturing,
                        frame_count,
                        frames,
                    ]
                )
            ]
        },
    )
    T.link((behavior, "capturing"), (capturing, "value"))
    T.link((behavior, "frame_count"), (frame_count, "value"))
    # T.dlink((behavior, "frames"), (frames, "children"))
    box.behaviors = {**box.behaviors, "graph_image": behavior}
    box.graph_ui = {**box.graph_ui, "graph_image": ui}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_graph_image(fg, box)
    graph_image = fg.behaviors[-1]

    # change the number of images to capture to 3
    graph_image.frame_count = 3

### `GraphData`

The current graph data can be streamed back to the kernel.

In [ ]:
def add_graph_data(fg, box):
    behavior = B.GraphData()
    capturing = W.ToggleButton(description="capturing")
    source_count = W.IntSlider()
    W.Text()
    ui = U.make_a_collapsible_picker(
        "graph: data",
        children={
            "capture": [
                W.VBox(
                    [
                        capturing,
                        source_count,
                    ]
                )
            ]
        },
    )
    T.link((behavior, "capturing"), (capturing, "value"))
    T.link((behavior, "source_count"), (source_count, "value"))
    box.behaviors = {**box.behaviors, "graph_data": behavior}
    box.graph_ui = {**box.graph_ui, "graph_data": ui}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_graph_data(fg, box)

    graph_data = fg.behaviors[-1]

    # change the number of source to capture to 3
    graph_data.source_count = 3

## Node Behaviors

#### Behavior Attributes

Behavior attributes support either a `Column` or a `Nunjunks` template.

`Nunjucks` take the form of
[nunjucks templates](https://mozilla.github.io/nunjucks/templating.html), this allows
for calculating dynamic values on the client.

The syntax is intentionally very similar to
[jinja2](https://jinja.palletsprojects.com/en/3.1.x/templates), and a number of
[extra template functions](#Templates) are provided

Inside of a template, one can use:

- `node`
  - this will have all of the named columns available to it
- `graphData`
  - `nodes`
  - `links`
    - `source` and `target` as realized nodes

With these, and basic template tools, one can generate all kinds of interesting effects.
For the example data above, try these color templates:

- color by group
  > ```python
  > {{ ["red", "yellow", "blue", "orange", "purple", "magenta"][node.group] }}
  > ```
- color by out-degree
  > ```python
  > {% set n = 0 %}
  > {% for link in graphData.links %}
  >   {% if link.source.id == node.id %}{% set n = n + 1 %}{% endif %}
  > {% endfor %}
  > {% set c = 256 * (7-n) / 7 %}
  > rgb({{ c }},0,0)
  > ```

### `NodeSelection`

The `NodeSelection` behavior allows for selecting one or more nodes from the browser, or
updating from the kernel.

By default, this is purely transient data, but can be populated with data accessible
when recording with [`GraphData`](#GraphData) by setting a `column_name`.

In [ ]:
def add_node_selection(fg, box):
    selection = B.NodeSelection()

    ui_column_name = W.Text(placeholder="update column")
    T.dlink(
        (ui_column_name, "value"),
        (selection, "column_name"),
        lambda x: x.strip() if x.strip() else None,
    )

    ui_selection = W.TagsInput(
        placeholder="select some node indices",
        allowed_tags=[*sorted(fg.source.nodes.id)],
    )

    def on_source_change(change):
        ui_selection.value = []
        ui_selection.allowed_tags = sorted(fg.source.nodes.id)

    def on_node_selected(change):
        if not change.new:
            ui_selection.value = []
        selected_ids = set(fg.source.nodes.id[[*change.new]])
        if selected_ids.symmetric_difference(ui_selection.value):
            ui_selection.value = tuple(selected_ids)

    def on_tag_selected(change):
        if not change.new:
            selection.selected = []
        all_ids = [*fg.source.nodes.id]
        selected_indices = {all_ids.index(id_) for id_ in change.new}
        if selected_indices.symmetric_difference(selection.selected):
            selection.selected = tuple(selected_indices)

    fg.observe(on_source_change, ["source"])
    selection.observe(on_node_selected, ["selected"])
    ui_selection.observe(on_tag_selected, ["value"])

    ui = W.HBox([ui_column_name, ui_selection])

    box.behaviors = {**box.behaviors, "node_selection": selection}
    box.node_ui = {**box.node_ui, "node_selection": ui}
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_node_selection(fg, box)

### `NodeShapes`

> Node Styling (e.g., color, size, shape)

Node colors, shape, and size can be set using the `NodeShapes`, more information on the
[⭐ Shapes](./Shapes.ipynb) notebook.

In [ ]:
if __name__ == "__main__":
    shape_ui = S.make_full_node_shape_ui(fg, box)

> 💡 Note that the nodes changed colors. Click a node to select it, or use
> <kbd>ctrl</kbd> or <kbd>shift</kbd> to select multiple nodes.

The selection is handed back from the client, and can be used with other widgets.

### `NodeTooltip`

Node labels can be revealed when hovering over the node. By default the node's `id`
column will be used.

In [ ]:
def add_node_tooltips(fg, box, column_name="id"):
    behavior = B.NodeTooltip()
    ui_label_column = W.Dropdown(
        options=list(fg.source.nodes.columns), value=column_name
    )
    U.subscribe_to_columns(fg, "nodes", ui_label_column, "options")
    ui_label_template = W.Textarea()
    ui_label_template_enabled = W.Checkbox(description="enabled?")
    T.dlink((ui_label_column, "value"), (behavior, "label"), lambda x: B.Column(x))
    T.dlink(
        (ui_label_template, "value"),
        (behavior, "label"),
        lambda x: B.Nunjucks(x) if ui_label_template_enabled.value else None,
    )
    ui = U.make_a_collapsible_picker(
        "node: tooltip",
        {
            "off": [],
            "column": [ui_label_column],
            "template": [W.VBox([ui_label_template_enabled, ui_label_template])],
        },
    )
    label = "node: labels"
    box.node_ui = {**box.node_ui, label: ui}
    U.make_link_dropdown_responsive(behavior, label, ui, box)
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_node_tooltips(fg, box)

#### Label templates

`NodeTooltips` accepts [templates](#Behavior-Attributes). The resulting value may be
plain strings or HTML.

Here are some examples, again for the example data:

- just a header
  > ```html
  > <h1>{{ node.id }}</h1>
  > ```
- a table
  > ```html
  > <table>
  >  <tr><th>id</th><th>group</th></td>
  >  {% for link in graphData.links %}
  >  {% if link.source.id == node.id %}
  >  <tr><td>{{ link.target.id }}</td><td>{{ link.target.group }}</td>
  >  {% endif %}
  >  {% endfor %}
  > </table>
  > ```

## Link Behaviors

### `LinkSelection`

Like the `NodeSelection` behavior, this allows for selecting one or more links from the
browser, or updating from the kernel.

In [ ]:
def add_link_selection(fg, box):
    selection = B.LinkSelection()

    ui_column_name = W.Text(placeholder="update column")
    T.dlink(
        (ui_column_name, "value"),
        (selection, "column_name"),
        lambda x: x.strip() if x.strip() else None,
    )

    ui_selection = W.IntsInput(
        placeholder="select some link indices",
        allowed_tags=[*range(len(fg.source.links))],
    )

    def on_source_change(change):
        ui_selection.value = []
        ui_selection.allowed_tags = [*range(len(fg.source.links))]

    fg.observe(on_source_change, ["source"])
    ui = W.HBox([ui_column_name, ui_selection])
    box.behaviors = {**box.behaviors, "link_selection": selection}
    box.link_ui = {**box.link_ui, "link_selection": ui}
    T.link((selection, "selected"), (ui_selection, "value"))
    return fg, box

In [ ]:
if __name__ == "__main__":
    add_link_selection(fg, box)

> 💡 Note that the links changed colors. Click a link to select it, or use
> <kbd>ctrl</kbd> or <kbd>shift</kbd> to select multiple links.

The selection is handed back from the client, and can be used with other widgets.

### `LinkShapes`

Link `widths` and `colors` can also be configured by `Column`, a `Nunjucks` template, or
by literal values. In addition, the `shapes` trait can carry one or more
[text shapes](./LinkText.ipynb).

In [ ]:
def add_link_shapes(fg, box):
    return U.add_behavior(B.LinkShapes(), fg, box, "link: shapes", "links")

In [ ]:
if __name__ == "__main__":
    add_link_shapes(fg, box)

### `LinkTooltip`

Link labels are revealed when hovered. Their value can be formatted using `HTML`, using
`Column`, a `Nunjucks` template, or by literal values.

In [ ]:
def add_link_tooltips(fg, box):
    return U.add_behavior(B.LinkTooltip(), fg, box, "link: tooltips", "links")

In [ ]:
if __name__ == "__main__":
    add_link_tooltips(fg, box)

### `LinkArrows`

Display the directionality (and potentially other properties) of a link using an
`arrow`.

In [ ]:
def add_link_arrows(fg, box):
    return U.add_behavior(B.LinkArrows(), fg, box, "link: arrows", "links")

In [ ]:
if __name__ == "__main__":
    add_link_arrows(fg, box)

### `Particles`

Can be used to dynamically illustrate properties on a `link`, e.g., directionality, or
some quantitative measure through their properties:

- `color`: can be specified `Column`, a `Nunjucks` template, or by literal string value.
- `speed`: can be specified `Column`, a `Nunjucks` template, or by literal numeric
  value, ideally `0.0 < speed < ~0.1`.
- `width`: the size of the particles, `ideally 0.0 < width < ~5`
- `density`: the number of particles on a link, ideally `greater than 0.0`.

In [ ]:
def add_link_particles(fg, box):
    return U.add_behavior(B.LinkParticles(), fg, box, "link: particles", "links")

In [ ]:
if __name__ == "__main__":
    add_link_particles(fg, box)

## Templates

In addition to the
[built-in `nunjucks` globals, templates, and tags](https://mozilla.github.io/nunjucks/templating.html),
the following
[JS `Math`](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math)
constants and functions are also exposed.

### Collections

#### `where`

Return all items that match a given attribute, e.g. for `NodeSizes.template` to scale a
node by out-degree.

```
{{ graphData.links | where('source', node) | count }}
```

### Time

#### `now`

Returns the current high-precision time from
[`performance.now`](https://developer.mozilla.org/en-US/docs/Web/API/Performance/now)

### Constants

> All are referenced directly, e.g., `{{ E }}`

|   Name    |   Symbol / Function    |        Value        |
| :-------: | :--------------------: | :-----------------: |
|    `E`    |         $$e$$          |  $$\approx 2.718$$  |
|   `PI`    |        $$\pi$$         | $$\approx 3.14159$$ |
|   `LN2`   |       $$\ln(2)$$       |  $$\approx 0.693$$  |
|  `LN10`   |      $$\ln(10)$$       |  $$\approx 2.302$$  |
|  `LOG2E`  |     $$\log_{2}e$$      |  $$\approx 1.442$$  |
| `LOG10E`  |     $$\log_{10}e$$     |  $$\approx 0.434$$  |
| `SQRT1_2` | $$\sqrt{\frac{1}{2}}$$ |  $$\approx 0.707$$  |
|  `SQRT2`  |      $$\sqrt{2}$$      |  $$\approx 1.414$$  |

### Unary Math Functions

> All accept a single numeric argument, e.g., `{{ acos(node.value) }}`

|   Name   |              Function               | Description                                                                                                |
| :------: | :---------------------------------: | :--------------------------------------------------------------------------------------------------------- |
|  `acos`  |           $$\arccos(x)$$            | inverse cosine ($x$ in radians)                                                                            |
| `acosh`  |    $$\ln( x + \sqrt{x^2 - 1} )$$    | inverse hyperbolic cosine                                                                                  |
|  `asin`  |           $$\arcsin(x)$$            | inverse sine ($x$ in radians)                                                                              |
| `asinh`  |      $$\ln{(x+\sqrt{x^2+1})}$$      | inverse hyperbolic sine                                                                                    |
|  `atan`  |           $$\arctan(x)$$            | inverse tangent ($x$ in radians)                                                                           |
| `atanh`  | $$\frac{1}{2}\ln(\frac{1+x}{1-x})$$ | inverse hyperbolic tangent                                                                                 |
|  `cbrt`  |           $$\sqrt[3]{x}$$           | cube root                                                                                                  |
|  `ceil`  |         $$\lceil{x}\rceil$$         | rounds up and returns the smaller integer greater than or equal to a given number                          |
|  `cos`   |             $$\cos(x)$$             | cosine ($x$ in radians)                                                                                    |
|  `cosh`  |      $$\frac{e^x+e^{-x}}{2}$$       | hyperbolic cosine                                                                                          |
|  `exp`   |              $$e^{x}$$              | $e$ raised to the given power                                                                              |
| `expm1`  |             $$e^x - 1$$             | $e$ raised to the power of a number minus 1                                                                |
| `floor`  |        $$\lfloor{x}\rfloor$$        | rounds down and returns the largest integer less than or equal to a given number                           |
| `fround` |                                     | the nearest 32-bit single precision float representation of a number                                       |
|  `log`   |             $$\ln{x}$$              | natural logarithm (base $e$) of a number                                                                   |
| `log10`  |          $$\log_{10}{x}$$           | base 10 logarithm of a number                                                                              |
| `log1p`  |           $$\ln{(1+x)}$$            | natural logarithm of $1 + x$                                                                               |
|  `log2`  |           $$\log_{2}{x}$$           | base 2 logarithm of a number                                                                               |
|  `sign`  |                                     | returns $1$ or $-1$, indicating the sign of the number passed as argument, $0$ or $-0$ are returned as-is. |
|  `sin`   |             $$\sin(x)$$             | sine ($x$ in radians)                                                                                      |
|  `sinh`  |      $$\frac{e^x-e^{-x}}{2}$$       | hyperbolic sine                                                                                            |
|  `sqrt`  |            $$\sqrt{x}$$             | square root                                                                                                |
|  `tan`   |             $$\tan{x}$$             | tangent ($x$ in radians)                                                                                   |
|  `tanh`  |  $$\frac{e^x-e^{-x}}{e^x+e^{-x}}$$  | hyperbolic tangent                                                                                         |
| `trunc`  |                                     | truncate float to the integer part, e.g., `trunc(41.84) = 41`                                              |

### Binary Math Functions

> All accept exactly two numerical arguments, e.g., `{{ imul(node.value, 1) }}`

| Function Name |      Symbol       |                    Description                     |
| :-----------: | :---------------: | :------------------------------------------------: |
|    `atan2`    | $$\arctan(y, x)$$ |    arctangent of the quotient of its arguments     |
|    `imul`     |                   | C-like 32-bit multiplication of the two parameters |

### N-ary Math Functions

> All accept _either_ a `list of numbers` or `n-numeric arguments`, e.g.,
> `{{ min([4,5,6]) }}`, `{{ min(3,2,1) }}`

| Function Name |                 Symbol                 |        Description        |
| :-----------: | :------------------------------------: | :-----------------------: |
|    `hypot`    | $$\sqrt{x_1^2 + x_2^2 + ... + x_n^2}$$ | n-dimensional hypothenuse |
|     `max`     |      $$\max(x_1, x_2, ..., x_n)$$      |       maximum value       |
|     `min`     |      $$\min(x_1, x_2, ..., x_n)$$      |       minimum value       |

In [ ]:
all_behaviors = [
    add_dataset,
    add_graph_colors,
    add_graph_image,
    add_graph_data,
    add_node_selection,
    add_node_tooltips,
    add_link_selection,
    add_link_tooltips,
    add_link_shapes,
    add_link_arrows,
    add_link_particles,
]